In [1]:
exec(open("./funcs/tool_funcs.py").read())


In [2]:
# For the widest path problem, we need a Max-Heap, rather than a Min-Heap. 
# We use some techniques to convert a Min-Heap into a Max-Heap.


def dijkstra_show_path_widest_path(adj_matrix, start, end):
 
    n = len(adj_matrix)
    distances = [0] * n
    previous_nodes = [-1] * n
    distances[start] = np.inf
    
    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
    priority_queue = [(-np.inf, start)]  # (distance, node)

 
    
    while priority_queue:
       
        
        # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
        current_distance, current_node = heapq.heappop(priority_queue)
        current_distance = -current_distance

        # Stop if we reached the destination node
        if current_node == end:
            break

        # If the distance is no longer optimal, skip
        if current_distance < distances[current_node]:
            continue

        # Explore neighbors
        for neighbor, weight in enumerate(adj_matrix[current_node]):
            if weight < np.inf:  
                distance = min(current_distance, weight)

                # Update distance if it's better
                if distance > distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node
                    
                    # For the widest path problem, we need a Max-Heap, rather than a Min-Heap.
                    heapq.heappush(priority_queue, (-distance, neighbor))

    # Reconstruct the path from end to start
    path = []
    current = end
    while current != -1:
        path.append(current)
        current = previous_nodes[current]
    path.reverse()

    # If the start node isn't reachable, return an empty path
    return path if path[0] == start else []

In [3]:
def translate_path(path, candidate_node_list):
    new_path = [candidate_node_list[i] for i in path]
    return new_path




In [4]:
def output_small_matrix(X_distance_matrix, candidate_node_list):
    small_matrix = np.zeros((len(candidate_node_list), len(candidate_node_list)))
    for i, ii in enumerate(candidate_node_list):
        for j, jj in enumerate(candidate_node_list):
            small_matrix[i,j] = X_distance_matrix[ii, jj]
    return small_matrix




In [5]:
def cal_candidate_node_list_widest_path(i, j, widest_path_matrix, X_distance_matrix):
    
    N = len(X_distance_matrix)
    
    remaining_list = [k for k in np.arange(N) if k != i and k != j]
    candidate_node_list = []
    candidate_node_list.append(i)
    for t in remaining_list:
        if widest_path_matrix[i,j] > min(widest_path_matrix[i,t], widest_path_matrix[t,j]):
            pass
        else:
            candidate_node_list.append(t)
    candidate_node_list.append(j)
 
    return candidate_node_list
        



In [6]:

def cal_path_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_widest_path(i, j, widest_path_matrix, X_distance_matrix)
    print("candidate_node_list: \n", candidate_node_list)
    print("Size of candidate_node_list: ", len(candidate_node_list))
   
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_widest_path(small_matrix, 0, len(candidate_node_list) - 1)
    path = translate_path(path, candidate_node_list)
    
    return path


In [7]:
# APPD stands for all points path distance (APPD).

def check_widest_path(i,j, X_distance_matrix, APPD_matrix, path):
    temp1= path[:-1]
    temp2= path[1:]
    llll = [X_distance_matrix[temp1[i],temp2[i]] for i in range(len(temp1))]
    ooo = np.min(llll)
    return ooo == APPD_matrix[i,j]

def check_if_temp_path_already_in(all_paths_list, temp_path):
    aa = set(temp_path)
    for bb in all_paths_list:
        bb = set(bb)
        if aa == bb:
            return True
    return False

def cal_all_paths_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix):
    candidate_node_list = cal_candidate_node_list_widest_path(i, j, widest_path_matrix, X_distance_matrix)
 
    small_matrix = output_small_matrix(X_distance_matrix, candidate_node_list)
    path = dijkstra_show_path_widest_path(small_matrix, 0, len(candidate_node_list) - 1)

    all_paths_list = []
    all_paths_list.append(path)

    K = len(candidate_node_list)
    temp_list = [q for q in range(K) if q not in path]

    for m in temp_list:
        temp_path1 = dijkstra_show_path_widest_path(small_matrix, 0, m)
        temp_path2 = dijkstra_show_path_widest_path(small_matrix, m, K - 1)
        temp_path =  temp_path1[:-1] + temp_path2
        if not check_if_temp_path_already_in(all_paths_list, temp_path):
            all_paths_list.append(temp_path)

    all_paths_list = [translate_path(ppp, candidate_node_list) for ppp in all_paths_list]    

    return all_paths_list

In [8]:
# N = 100

# X_distance_matrix = create_distance_matrix(N)


X_distance_matrix = np.loadtxt("./data/X_100_distance_matrix.csv", delimiter=",")


In [9]:

# For the widest path problem, a node's bandwidth to itself should be infinity.


N = len(X_distance_matrix)
for i in range(N):
    X_distance_matrix[i,i] = np.inf


In [10]:


 
widest_path_matrix = floyd_warshall_widest_path(X_distance_matrix)
 

In [11]:
i = 10
j = 50

path1 = dijkstra_show_path_widest_path(X_distance_matrix, i, j)  
path2 = cal_path_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix) 
print("Paths:")
print(path1)
print(path2)


    

candidate_node_list: 
 [10, 0, 1, 2, 4, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 27, 29, 31, 32, 34, 35, 36, 37, 39, 40, 41, 42, 43, 45, 47, 48, 49, 51, 52, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 74, 75, 77, 78, 80, 81, 83, 84, 87, 88, 89, 91, 93, 94, 95, 96, 97, 98, 99, 50]
Size of candidate_node_list:  78
Paths:
[10, 0, 80, 50]
[10, 0, 80, 50]


In [12]:
check_widest_path(i,j, X_distance_matrix, widest_path_matrix, path2)



True

In [13]:
all_paths_list = cal_all_paths_warm_start_widest_path(i, j, widest_path_matrix, X_distance_matrix)

In [14]:
for path_i in all_paths_list:
    aaa = check_widest_path(i,j, X_distance_matrix, widest_path_matrix, path_i)
    print(path_i, aaa)



[10, 0, 80, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 65, 22, 70, 1, 34, 69, 29, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 43, 2, 64, 98, 65, 22, 96, 9, 72, 27, 7, 77, 81, 50] True
[10, 0, 80, 54, 64, 98, 4, 8, 69, 29, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 72, 6, 12, 32, 45, 9, 29, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 7, 77, 81, 50] True
[10, 0, 80, 54, 64, 98, 4, 8, 65, 22, 96, 9, 72, 27, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 72, 27, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 65, 94, 36, 11, 15, 80, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 72, 27, 13, 47, 87, 21, 6, 12, 32, 45, 9, 29, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 65, 94, 36, 11, 15, 80, 74, 71, 51, 2, 64, 98, 65, 22, 96, 9, 72, 27, 7, 77, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 65, 94, 36, 37, 95, 88, 24, 56, 16, 42, 81, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 7, 17, 0, 80, 50] True
[10, 0, 80, 54, 12, 32, 45, 9, 29, 7, 77, 19, 18, 

In [15]:
len(all_paths_list)

49